In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
import re
import sys
import pickle
import pandas as pd
import numpy as np
from collections import Counter

In [44]:
df = pd.read_csv("/content/drive/MyDrive/datasets/flipkart_com-ecommerce_sample.csv")

In [45]:
df.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [46]:
df["product_category_tree"][0]

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

In [47]:
useful_data = df[["product_name","product_category_tree", "description", "brand"]]

In [48]:
useful_data.head()

,product_name,product_category_tree,description,brand
0,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha
1,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor
2,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",Key Features of AW Bellies Sandals Wedges Heel...,AW
3,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",Key Features of Alisha Solid Women's Cycling S...,Alisha
4,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",Specifications of Sicons All Purpose Arnica Do...,Sicons


In [49]:
len(useful_data)

20000

In [50]:
data = useful_data.copy()

In [51]:
data["description"][1]

"FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price: Rs. 22,646 • Fine deep seating experience • Save Space with the all new click clack Sofa Bed • Easy to fold and vice versa with simple click clack mechanism • Chrome legs with mango wood frame for long term durability • Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience • A double bed that can easily sleep two,Specifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Modern 

In [52]:
data["new_desc"] = data["description"]

In [53]:
data["new_desc"] = data["new_desc"].str.replace(r'Rs.', '',regex=True)

In [54]:
data["new_desc"][1]

"FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price:  22,646 • Fine deep seating experience • Save Space with the all new click clack Sofa Bed • Easy to fold and vice versa with simple click clack mechanism • Chrome legs with mango wood frame for long term durability • Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience • A double bed that can easily sleep two,Specifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Modern Fil

In [55]:
data["new_desc"] = data["new_desc"].str.replace(r'\u2022+?',r'.',regex=True)

In [56]:
data["new_desc"] = data["new_desc"].str.replace(r'(?<=\d),(?=\d)', r'',regex=True)

In [57]:
data["new_desc"][1]

"FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price:  22646 . Fine deep seating experience . Save Space with the all new click clack Sofa Bed . Easy to fold and vice versa with simple click clack mechanism . Chrome legs with mango wood frame for long term durability . Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience . A double bed that can easily sleep two,Specifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Modern Fill

In [58]:
data["new_desc"] = data["new_desc"].str.replace(r'(?<=[:;,(-])(?=[^\s])', r' ',regex=True)

In [60]:
data["new_desc"] = data["new_desc"].str.replace(r'(?<=[\w])([!):;,-])', r' \1',regex=True)

In [61]:
data["new_desc"][1]

"FabHomeDecor Fabric Double Sofa Bed ( Finish Color - Leatherette Black Mechanism Type - Pull Out ) Price :  22646 . Fine deep seating experience . Save Space with the all new click clack Sofa Bed . Easy to fold and vice versa with simple click clack mechanism . Chrome legs with mango wood frame for long term durability . Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience . A double bed that can easily sleep two , Specifications of FabHomeDecor Fabric Double Sofa Bed ( Finish Color - Leatherette Black Mechanism Type - Pull Out ) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Mode

In [73]:
data["new_desc"] = data["new_desc"].str.replace(r'\s{2,}', r' ',regex=True)

In [79]:
data["new_desc"] = data["new_desc"].str.replace(r'(?<=[\w\W])([\n+])', r' ',regex=True)

In [80]:
data["new_desc"][93]

'Vizion 8503B - 1RED Cold Light Digital Watch - For Boys , Girls Price : 595 Brand : Vizion , Sports Series watch for Kids , 1 year manufacturer warranty , Digital display , Automatic movement , Metal Case and solid body , Dial shape : Round , Dial dimension : 22mm , Strap material : Silicone , Strap width : 20mm , Water resistant , Night display cold backLight , Daily Alarm , Hour minute second , date week display , 12/24 hour format , Attractive brand packaging - Tuff plastic box , Note : In case of non - receipt of warranty card , please intimate us within 48 hours , Disclaimer : Product color may slightly vary due to photographic lighting sources or your monitor settings ( The image has been enlarged for better viewing ) Brand : Vizion , Sports Series watch for Kids , 1 year manufacturer warranty , Digital display , Automatic movement , Metal Case and solid body , Dial shape : Round , Dial dimension : 22mm , Strap material : Silicone , Strap width : 20mm , Water resistant , Night d

In [81]:
data["new_desc"][0]

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy , Red , Navy , Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water , Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [82]:
data["new_desc"][2]

'Key Features of AW Bellies Sandals Wedges Heel Casuals , AW Bellies Price : 499 Material : Synthetic Lifestyle : Casual Heel Type : Wedge Warranty Type : Manufacturer Product Warranty against manufacturing defects : 30 days Care instructions : Allow your pair of shoes to air and de - odorize at regular basis ; use shoe bags to prevent any stains or mildew ; dust any dry dirt from the surface using a clean cloth ; do not use polish or shiner , Specifications of AW Bellies General Ideal For Women Occasion Casual Shoe Details Color Red Outer Material Patent Leather Heel Height 1 inch Number of Contents in Sales Package Pack of 1 In the Box One Pair Of Shoes'

In [ ]:
# data["new_desc"] = data["new_desc"].str.replace(r'[^\.a-zA-Z]\s+?',r'',regex=True)

In [ ]:
# data["new_desc"] = data["new_desc"].str.replace(r'.',r' .',regex=True)

In [24]:
# data["new_desc"] = data["new_desc"].str.replace(r'(',r'( ',regex=True)

In [25]:
# data["new_desc"] = data["new_desc"].str.replace(r')',r' )',regex=True)

In [26]:
# data["new_desc"] = data["new_desc"].str.replace(r':',r' : ',regex=True)

In [27]:
# data["new_desc"] = data["new_desc"].str.replace(r'(?<=[,])(?=[^\d])', r' ',regex=True)

In [28]:
# data["new_desc"] = data["new_desc"].str.replace(r'(?<=\d),(?=\d)', r'',regex=True)

In [13]:
with open('/content/drive/MyDrive/datasets/temp.json', 'w') as f:
    f.write(useful_data.to_json(orient='records', lines=True))

In [16]:
import sys


def get_file_list_from_dir(datadir):
    data = []
    i = 0
    with open(datadir) as f:
        for line in f:
            data.append(line)
            sys.stdout.write(
                '%d line processed\r' % (i)
            )
            i += 1

    return data


def split(file_list):
    num = len(file_list)
    train = num // 10 * 8
    valid = train + (num - train) // 2
    training = file_list[:train]
    validing = file_list[train:valid]
    testing = file_list[valid:]
    return training, validing, testing


def write_files(filename, data):
    f = open(filename, "w")
    f.writelines(data)
    f.flush()
    f.close()


if __name__ == "__main__":
    path = '/content/drive/MyDrive/datasets/'
    train, valid, test = split(get_file_list_from_dir(path + 'temp.json'))
    filename = path + 'train_wiki_P.json'
    write_files(filename, train)
    print('Finished Product train')
    filename = path + 'valid_wiki_P.json'
    write_files(filename, valid)
    print('Finished Product valid')
    filename = path + 'test_wiki_P.json'
    write_files(filename, test)
    print('Finished Product test')

Finished Product train
Finished Product valid
Finished Product test


In [41]:
import re
line = ':Fabric -Double ;Sofa Bed (Finish Color - Leatherette: Black: Mechanism Type - Pull Out) Installation'
re.sub(r'(?<=[:;,(-])(?=[^\s])', r' ', line)

': Fabric - Double ; Sofa Bed ( Finish Color - Leatherette: Black: Mechanism Type - Pull Out) Installation'

In [42]:
re.sub(r'(?<=[\w])([!):;,-])', r' \1', line)

':Fabric -Double ;Sofa Bed (Finish Color - Leatherette : Black : Mechanism Type - Pull Out ) Installation'

In [77]:
txt = 'The image has been enlarged for better viewing )\nBrand : Vizion , Sports Series watch for Kids'

In [78]:
re.sub(r'(?<=[\w\W])([\n+])', r' ', txt)

'The image has been enlarged for better viewing ) Brand : Vizion , Sports Series watch for Kids'